In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import csv
import datetime

# Initialize Selenium WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
wait = WebDriverWait(driver, 20)

# Target URL
url = "https://www.bovada.lv/sports/esports/dota-2"
driver.get(url)

try:
    # Wait and find the email field, then send the email
    wait.until(EC.presence_of_element_located((By.ID, "email"))).send_keys("nickapch@gmail.com")
    
    # Wait and find the password field, then send the password
    wait.until(EC.presence_of_element_located((By.ID, "login-password"))).send_keys("P@r0lbovada")
    
    # Wait and find then click the login button
    wait.until(EC.element_to_be_clickable((By.ID, "login-submit"))).click()
except Exception as e:
    print('Login window didn\'t appear or other login issue occurred:', e)

# Wait for the login process to complete and for the next page to load
wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div.grouped-events")))

# Extracting today's date to include in the CSV
today_date = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')

# Initialize CSV file writing
with open('betting_odds.csv', mode='a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    if file.tell() == 0:
        writer.writerow(['Date', 'League Name', 'Team 1', 'Odd 1', 'Team 2', 'Odd 2', "Match Date"])
    
    # Extract all league sections
    leagues = driver.find_elements(By.CSS_SELECTOR, "div.grouped-events")
    for league in leagues:
        league_name = league.find_element(By.CSS_SELECTOR, "a.league-header-collapsible__description").text
        matches = league.find_elements(By.CSS_SELECTOR, "section.coupon-content")
        
        for match in matches:
            try:
                # Teams and Odds handling with checks for elements' presence
                team_names = [team.text for team in match.find_elements(By.CSS_SELECTOR, "div.competitors h4.competitor-name span.name")]
                team1_name, team2_name = (team_names + ['N/A', 'N/A'])[:2]
                
                odds = [odd.text for odd in match.find_elements(By.CSS_SELECTOR, "button.bet-btn span.bet-price")]
                odd1, odd2 = (odds + ['N/A', 'N/A'])[2:4]

                # Time and Date handling
                full_date = "Date not found"
                match_date_span = match.find_elements(By.CSS_SELECTOR, "span.period.hidden-xs")
                if match_date_span:
                    match_date_text = match_date_span[0].text.strip()
                    match_time_elements = match_date_span[0].find_elements(By.TAG_NAME, "time")
                    if match_time_elements:
                        match_time = match_time_elements[0].text.strip()
                        match_date = match_date_text.replace(match_time, '').strip()
                        full_date = f"{match_date} {match_time}"
                    else:
                        full_date = match_date_text
                
                print(f'Writing {[today_date, league_name, team1_name, odd1, team2_name, odd2, full_date]}')
                writer.writerow([today_date, league_name, team1_name, odd1, team2_name, odd2, full_date])
            except Exception as e:
                print("An error occurred while processing a match:", e)

driver.quit()


Login window didn't appear or other login issue occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x007AB133+46099]
	(No symbol) [0x0073C461]
	(No symbol) [0x00630C57]
	(No symbol) [0x00670E3D]
	(No symbol) [0x00670EEB]
	(No symbol) [0x006ABE82]
	(No symbol) [0x00691044]
	(No symbol) [0x006A9E15]
	(No symbol) [0x00690D96]
	(No symbol) [0x0066595D]
	(No symbol) [0x0066651D]
	GetHandleVerifier [0x00A64BA3+2903683]
	GetHandleVerifier [0x00AA261B+3156219]
	GetHandleVerifier [0x0084552B+677899]
	GetHandleVerifier [0x0084B6FC+702940]
	(No symbol) [0x00745A24]
	(No symbol) [0x00741A68]
	(No symbol) [0x00741BFC]
	(No symbol) [0x00734750]
	BaseThreadInitThunk [0x77097BA9+25]
	RtlInitializeExceptionChain [0x778EBE3B+107]
	RtlClearBits [0x778EBDBF+191]

Writing ['2024-04-23 17:47', 'Dota 2 - ESL One Birmingham', 'BB Team', 'N/A', 'Shopify Rebellion', 'N/A', 'LIVE']
Writing ['2024-04-23 17:47', 'Dota 2 - ESL One Birmingham', 'G2.IG', 'N/A', 'Talon', 'N/A', 'LIVE']
Writing ['2024-04-23 17:47', 'D